In [435]:
from hashlib import sha256
import struct
from base58
import random
from collections import deque
from functools import lru_cache

b58e = lambda x: base58.b58encode(x).decode('ascii')

EMPTY = b'\x00' * 32

@lru_cache(maxsize=32)
def empty(level):
    if level == 0:
        return EMPTY
    value = empty(level - 1)
    return h(value, value)

def h(*args):
    sha = sha256()
    for a in args:
        sha.update(a)
    return sha.digest()

class MerkleTree:
    def __init__(self, leaves, root=None):
        assert(len(set(leaves)) == len(leaves))
        self.free_list = deque()
        self.leaf_nodes = [TreeNode(h(x)) for x in leaves]
        self.leaf_dict = dict(zip(list(map(h, leaves)), self.leaf_nodes))
        self.root = self.build_root()
        
    def build_root(self):
        tree = deque(zip(self.leaf_nodes, [0] * len(self.leaf_nodes)))
        while len(tree) > 1:
            left, i = tree.popleft()
            if tree[0][1] != i:
                right = TreeNode(empty(i))
            else:
                right, _ = tree.popleft()
            parent = TreeNode(h(left.key, right.key), left, right)
            tree.append((parent, i + 1))
        return tree[0][0]
    
    def get_root(self):
        return b58e(self.root.key)
    
    def get_proof(self, leaf):
        if not isinstance(leaf, TreeNode):
            node = self.leaf_dict[leaf]
        else:
            assert(leaf.left == None)
            assert(leaf.right == None)
            node = leaf
        proof = []
        while node.parent is not None:
            key = node.key
            node = node.parent
            if node.left.key == key:
                proof.append((node.right.key, 1))
            else:
                proof.append((node.left.key, 0))
        return list(zip(*proof))
    
    def verify_proof(self, proof, path, leaf):
        key = leaf 
        for node, side in zip(proof, path):
            if side == 0:
                key = h(node, key)
            else:
                key = h(key, node)
        return key == self.root.key 
    
    def append_empty_leaf(self):
        node = self.leaf_nodes[-1]
        n = len(self.leaf_nodes)
        i = 0
        new_leaf = TreeNode(EMPTY)
        ptr = new_leaf 
        while n % 2 == 0 and node.parent is not None:
            i += 1
            n //= 2
            ptr = TreeNode(empty(i), ptr, TreeNode(empty(i-1)))  
            node = node.parent 
        key = h(node.key, ptr.key)
        if node.parent is None:
            parent = TreeNode(key, node, ptr)
        else:
            node.parent.key = key
            assert(node.parent.right.key == ptr.key)
            node.parent.right = ptr
            ptr.parent = node.parent 
        node = node.parent
        while node.parent is not None:
            key = node.key
            node = node.parent
            if node.left.key == key:
                node.key = h(key, node.right.key)
            else:
                node.key = h(node.left.key, key)
        self.root = node
        self.free_list.append(len(self.leaf_nodes))
        self.leaf_nodes.append(new_leaf)
    
    def add_leaf(self, leaf):
        if len(self.free_list) == 0:
            self.append_empty_leaf()
        leaf_node = self.leaf_nodes[self.free_list.popleft()]
        node = leaf_node 
        leaf_key = h(leaf)
        assert(leaf_key not in self.leaf_dict)
        node.key = leaf_key 
        while node.parent is not None:
            key = node.key 
            node = node.parent
            if node.left.key == key:
                node.key = h(key, node.right.key)
            else:
                node.key = h(node.left.key, key)
        self.leaf_dict[leaf_key] = leaf_node
            
    def remove_leaf(self, i):
        node = self.leaf_nodes[i] 
        key = node.key
        if key == EMPTY:
            return None
        node.key = EMPTY 
        while node.parent is not None:
            node = node.parent
            if node.left.key == key:
                node.key = h(key, node.right.key)
            else:
                node.key = h(node.left.key, key)
        self.free_list.appendleft(i)
        del self.leaf_dict[key]
        return key
    
    def remove_leaf_by_key(self, key):
        if key != EMPTY:
            indices = [j for (j, node) in enumerate(self.leaf_nodes) if node.key == key]
            if indices:
                return self.remove_leaf(indices[0])
        return None

    
class TreeNode:
    def __init__(self, key, left=None, right=None):
        self.key = key
        self.left = left
        self.right = right
        self.parent = None
        if left:
            left.parent = self
        if right:
            right.parent = self

leaves = [struct.pack("<Q", random.randint(0, (1<<64) - 1)) for _ in range((2**18) - 1)]

In [457]:
%%time
mt = MerkleTree(leaves)
mt.get_root()

CPU times: user 3.74 s, sys: 66.9 ms, total: 3.81 s
Wall time: 3.81 s


'6j1fXA84313XStq3xfgDs1zTKMpjgRuscjEUGdVMrTta'

In [456]:
mt.append_empty_leaf()
print(mt.get_root())
mt.append_empty_leaf()
print(mt.get_root())

CHdBFhHQsGsFYU6dGsX7EmXZs85CfcQDRAUGGzcKtECt
CHdBFhHQsGsFYU6dGsX7EmXZs85CfcQDRAUGGzcKtECt


In [438]:
%%time
target = hash_value(random.choice(leaves))
proof, path = mt.get_proof(target)
list(map(b58e, proof))

CPU times: user 524 µs, sys: 1 µs, total: 525 µs
Wall time: 530 µs


['FaZUEm6qLPmzw9Gp7HNKYspxf6KhYuVcEUZbpWfwqLSk',
 '5FxCmTuWYFiF6e5VP9ReTUi94AsARvGeBczgRfBnraMc',
 'Ck3BQaUiYfsHKachsBBWZonZ22MHfRogNuwsEtrfMQko',
 '4rv7zCF4a8LP2KnYw7KjczirsX96o7D5hGhSbxxrva9q',
 'adQsRY5jS1YYwi93nJ1y4PmNtF6QRCjK9KUTvPTLiCV',
 '2wo2XtonXtPd9KGczVt2s1rd2M4396utwvZykvzTzoDj',
 'G8YZoqLYLBCaHXt25GyhsDJdvVCRTbM7nh5NNBVTDKGY',
 'H5pdpAhA97xT1S2om8Dzb2ykUit6CyPCohkveEREqgU3',
 '9TSGfopeiJN8N1eTQB6usnCmQwxfhKqKhjYULsQjf3oa',
 '6SCq3pkwUjERxfxowDVsW4x4PAV4SWWSrL95HanTXioH',
 'Se56hpW5HVxewj38LiNhPtzxMXHJsv65HGbnkbrHt5p',
 '9bcYWzBzEdZX7Z3LyaEL8KXPYQf5VA8aZCDVbugxjxZB',
 'DtG4Hg1trYKdVrMsxS6vTSZx7LyRs4Kep64Fet827Ruk',
 '2oRVJyg9VEspWcrmFaRZMKv4qeytFmt7PkmdruhRGXKQ',
 '4uurwJcW6aGuNj3BQ6ePypMDX2VVJTS1pLARsnEY5bfU',
 '5nGk5ocVBJFeqeXMWa63JCLxe29pbAo1yWXHXvbQx2Yf',
 '4zcrFUyN97VT8dMzVWCpzgAY4H3aTvvq3p7DCdd2WZoA',
 'DrxWpLfTxD2Gt4KirsvWWMT5w6Nj1S4PXpLRpvuCr9nT',
 'B6GvsF6KYiXQq8Nfwa6wwRrT8eWoNpdtF42mPSzZ2x4s']

In [439]:
%%time

mt.verify_proof(proof, path, target)

CPU times: user 49 µs, sys: 0 ns, total: 49 µs
Wall time: 53.2 µs


True

In [455]:
if mt.free_list:
    head = mt.free_list[0]
    print(head)
else:
    head = len(mt.leaf_nodes)
new_leaf = struct.pack("<Q", random.randint(0, (1<<64) - 1))
mt.add_leaf(new_leaf)

assert(mt.leaf_nodes[head].key == h(new_leaf))

In [441]:
mt.leaf_nodes[-1].key == h(new_leaf)

False

In [444]:
list(map(lambda x: x.key, mt.leaf_nodes[-5:]))

[b',\x1c\xb4\x1ca\xba\xb6\x1a:0\xf2\xb7\xce\xf6\xf0\xe0b\xa8W\xbd \x84-\x99\x08\xabF\xec\x1c\xd2#\x1f',
 b'\xaa\xb7\x1f.\x00a\xce2\xbf\xd0\x1c&\xe8\x17\xc9\x01\xdb\xe3\x01\x9f\xf0\x06\x8e\xb6t\xd8\xb7]J\x00\xac ',
 b'D\xf7%q.\xa5_\xc0h\xeaEY\xb9\\\x94\x1c\x89\xa9ek}\xa3\xd6\xac\x90?P\x9bl\x87\x91o',
 b'\xbe\xdbe\x97z\xce\r\\\xac[q\xff\x0ch2:1\\\x8bt6\x07j\x84\n\x8d7\x82\x944x3',
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00']

In [445]:
mt.get_proof(mt.leaf_nodes[-2])

[(b'D\xf7%q.\xa5_\xc0h\xeaEY\xb9\\\x94\x1c\x89\xa9ek}\xa3\xd6\xac\x90?P\x9bl\x87\x91o',
  b'\x1d&Yk\xe4\x11KeX\x9b6\xd1Z\xc4\x85\x0c\x8f]\xc2\xcf+\xa2YD\x11(\x88\x8c\xfab\x12\x96',
  b'\xf4\xe9\xadE\x18y\x91)7\x95!\x17\xe4qK@\x8d\x14\xef\xee\x18\xf9\xd5\x8d\x95\xc8\xd4\xee\xff\x7f\xc7\xf0',
  b'k\xd0\xb4\xeb\xf5\xa9#\x0b\x13ZAJHb \xb2\x05!\x88\xa3K\xc6\x9a:Hx>\xc0G\xcf\xab\x88',
  b'\xa6\x1a\xd2\x96z\xf3%\x10\xb1x\xf3\xe9A!\x85q\x19D\xc8\xe44e\xe3\xae\xaeEk\xed\xd3\xefIe',
  b'\xf2$\x03?T\x0bnK\xe5\x9e!a\x8d\x94\x02\x04\xc4M<\xa1\xcd.GD\xec\xc5\x9a\x0ba\x89\x14g',
  b'\x16\x7f\x9b\xa2E\x03Z\x1d\x81i\xf9f\xaa!\xd0\xaaK\xd8\xd92\xca\x9f\r2\xc5\xd1\xb8\n{\xf7\xadQ',
  b'f\\\x03\xbfw8\xbb\xce!m\xc0"v#\x1e\xef\xa2\x13\xd2l\x1dC\x82\xcei\xaa\xa0\x95\xc3\xe2+\xb1',
  b'\x10\x81n\x98OG\xd7\xe8S\x9e_g;\x19\xb8\xef\xab\xf8\x00i\xfbvyJwh\x1fr^5;\xd5',
  b'\x0bE\xe1,\xbaq0\xf6\x04"\xebe\x95\x0bk\xe5\x9d\x84\x1c|\xf7^\x00\xd8w\x84\xaaL\xba\xb0\xef\x1d',
  b"\xab!\xac\xba\x13\xb4[-\tO8\x00\xf3\x1b\x

In [446]:
mt.get_proof(mt.leaf_nodes[-1])

[(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  b"\xf5\xa5\xfdB\xd1j 0'\x98\xefn\xd3\t\x97\x9bC\x00=# \xd9\xf0\xe8\xea\x981\xa9'Y\xfbK",
  b'\xdbV\x11N\x00\xfd\xd4\xc1\xf8\\\x89+\xf3Z\xc9\xa8\x92\x89\xaa\xec\xb1\xeb\xd0\xa9l\xde`jt\x8b]q',
  b'\xc7\x80\t\xfd\xf0\x7f\xc5j\x11\xf1"7\x06X\xa3S\xaa\xa5B\xedc\xe4LK\xc1_\xf4\xcd\x10Z\xb3<',
  b'Sm\x98\x83\x7f-\xd1e\xa5]^\xea\xe9\x14\x85\x95Dr\xd5o$m\xf2V\xbf<\xae\x195*\x12<',
  b'\x9e\xfd\xe0R\xaa\x15B\x9f\xae\x05\xba\xd4\xd0\xb1\xd7\xc6M\xa6M\x03\xd7\xa1\x85JX\x8c,\xb8C\x0c\r0',
  b'\xd8\x8d\xdf\xee\xd4\x00\xa8uU\x96\xb2\x19B\xc1I~\x11L0.a\x18)\x0f\x91\xe6w)v\x04\x1f\xa1',
  b'\x87\xeb\r\xdb\xa5~5\xf6\xd2\x86g8\x02\xa4\xafYu\xe2%\x06\xc7\xcfLd\xbbk\xe5\xee\x11R\x7f,',
  b'&\x84dv\xfd_\xc5J]C8Qg\xc9QD\xf2d?S<\xc8[\xb9\xd1kx/\x8d}\xb1\x93',
  b'Pm\x86X-%$\x05\xb8@\x01\x87\x92\xca\xd2\xbf\x12Y\xf1\xefZ\xa5\xf8\x87\xe1<\xb2\xf0\tOQ\xe1',
  b'\xff\xff\n\xd7\